In [1]:
!pip install --upgrade google-cloud-datastore

  Using cached https://files.pythonhosted.org/packages/b8/6c/bedcab39e8dc969f7a48d058dbacd69fc07ce3f817a03de875902016f667/google_cloud_datastore-1.15.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ad/fc/6e8c449185cb8862af353c1164100ff75e32d55ba1de3baf9eaa01b7d2a9/google_cloud_core-1.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6c/13/c5ed31377b620158a487c5851343d3a59065954f83a34223c9b9087f7b8d/google_auth-1.28.1-py2.py3-none-any.whl
pandas-gbq 0.3.0 has requirement google-cloud-bigquery>=0.28.0, but you'll have google-cloud-bigquery 0.23.0 which is incompatible.
google-cloud-bigquery 0.23.0 has requirement google-cloud-core<0.24dev,>=0.23.1, but you'll have google-cloud-core 1.6.0 which is incompatible.
tensorboard 1.8.0 has requirement bleach==1.5.0, but you'll have ble

In [3]:
from google.cloud import datastore
import csv

datastore_client = datastore.Client()
with open('Friends.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    line_count = 0
    header=''
    tempdict = dict()
    for row in csvreader:
        if line_count == 0:
            header = row
            print header
            line_count += 1
            continue
            
        kind = 'friends'
        print('row',row)
        if row[0] not in tempdict:
            tempdict[row[0]] = list()
        tempdict[row[0]].append(int(row[1]))
        line_count += 1
        #for dev and testing
        if line_count == 20:
            break
            
    print("tempdict", tempdict)
    
    for k in tempdict.keys():
        with datastore_client.transaction():
            incomplete_key = datastore_client.key("friends")
            task = datastore.Entity(key=incomplete_key)
            task.update(
                {
                    "id": k,
                    "fids": tempdict[k],
                }
            )
            datastore_client.put(task)
    
    #for validation
    query = datastore_client.query(kind='friends')
    results = list(query.fetch())
    print results

['userId', 'fid']
('row', ['1', '328'])
('row', ['1', '234'])
('row', ['1', '91'])
('row', ['1', '592'])
('row', ['1', '489'])
('row', ['1', '356'])
('row', ['1', '655'])
('row', ['1', '130'])
('row', ['1', '158'])
('row', ['1', '256'])
('row', ['2', '539'])
('row', ['2', '654'])
('row', ['2', '619'])
('row', ['2', '77'])
('row', ['2', '667'])
('row', ['2', '81'])
('row', ['2', '55'])
('row', ['2', '155'])
('row', ['2', '350'])
('tempdict', {'1': [328, 234, 91, 592, 489, 356, 655, 130, 158, 256], '2': [539, 654, 619, 77, 667, 81, 55, 155, 350]})
[<Entity(u'friends', 5714489739575296L) {u'fids': [328L, 234L, 91L, 592L, 489L, 356L, 655L, 130L, 158L, 256L], u'id': '1'}>, <Entity(u'friends', 6228623363145728L) {u'fids': [539L, 654L, 619L, 77L, 667L, 81L, 55L, 155L, 350L], u'id': '2'}>]
